## Recipe of the Day Agent

The Recipe of the Day Agent is an orchestrated AI workflow designed to automatically generate, filter, enrich, and deliver the detailed recipe to users via email. It leverages multiple specialized sub-agents (or “tools”) and handoffs coordinated by a central Orchestrator Agent.

### Prerequisites:

1. You need a Gemini API key. This key should be set as an environment variable named GEMINI_API_KEY. 
2. You need a Sendgrid API key. This key should be set as an environment variable named SENDGRID_API_KEY. 
3. Save the sender email id and the receiver email id in SENDER_EMAIL_ID and RECEIVER_EMAIL_ID environment variables.

### Recipe Orchestrator Agent workflow:

1️⃣ Recipe Ideation Tool
- Generates 5 diverse recipe ideas
- Returns a Pydantic-based RecipeList model

2️⃣ Recipe Filter & Selector Agent
- Applies selection rules
- Chooses one recipe from the list

3️⃣ Recipe Details Agent
- Expands the selected recipe
- Adds ingredients, steps, cook time, etc.

4️⃣ Emailer Agent
- Generates email subject & HTML body
- Uses SendGrid API to deliver the email


In [41]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import os
import asyncio
from openai import AsyncOpenAI
from pydantic import BaseModel
from typing import Literal, List, Dict

In [71]:
load_dotenv(override=True)

True

In [47]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if sendgrid_api_key:
    print(f"Sendgrid API Key exists and begins {sendgrid_api_key[:3]}")
else:
    print("Sendgrid API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Sendgrid API Key exists and begins SG.


In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

In [53]:
class Recipe(BaseModel):
    title: str
    cuisine: Literal["Indian", "Italian", "Mexican", "British"]
    protein_source: str
    max_cook_time_minutes: int
    brief_description: str
    estimated_protein_gms: int

class RecipeList(BaseModel):
    recipes: List[Recipe]

In [27]:
instructions = """Follow below step carefully:
    1. MUST Generate a list of 5 diverse recipes.
    2. Each recipe MUST be vegeterian, protein-rich and be easy-to-cook. Strict adherence to the provided schema is required.
    3. Handoff all the generated recipes to the 'Recipe Filter and Selector' agent. The Recipe Filter and Selector will take care of filtering and selecting.
    Crucial Rules:
- You must generate the recipes first yourself and only then hand off to the 'Recipe Filter and Selector' agent"""

recipe_ideation_agent = Agent(
    name="Recipe Ideation Agent",
    instructions=instructions,
    model="gpt-4o-mini",
    output_type=RecipeList)

recipe_ideation_tool = recipe_ideation_agent.as_tool(
    tool_name="recipe_ideation_agent",
    tool_description="Recipe ideation agent tool")

In [26]:
filter_selection_instructions = """You are a recipe filtering and selecting agent.
    You receive the list of recipes as input.
    
    Your job is to filter recipes according to the following criteria:
    1. Must be vegetarian
    2. Must be protein-rich (≥ 15 grams of protein per serving)
    3. Must be under 30 minutes preparation time
    4. Must be Indian, Italian, Mexican or British cuisine

    From the filtered recipes, select 1 best recipe based on:
    1. Highest protein content
    2. Variety of cuisines
    3. Balanced ingredient list
"""

recipe_filter_selector_agent = Agent(
    name="Recipe Filter and Selector",
    instructions=filter_selection_instructions,
    model=gemini_model,
    output_type=Recipe
)

recipe_filter_selector_tool = recipe_filter_selector_agent.as_tool(
    tool_name="recipe_filter_selector_agent",
    tool_description="Recipe Filter And Selector Tool")

In [48]:
details_instructions = """You are a recipe details agent.
    You receive a recipe as input.
    
    Your job is to find the details of the recipe including instructions to cook, ingredients etc.

    Your final message should include:
    - **Title**: Name of the recipe
    - **Cuisine / Theme**
    - **Meal Type** (e.g. breakfast, lunch, dinner, dessert)
    - **Ingredients**
    - **Instructions**
    - **Nutrition Info**
    - **Shopping List**
    - **Chef’s Note**: one friendly line or tip from you as the orchestrator
"""

recipe_details_agent = Agent(
    name="Recipe Details Agent",
    instructions=details_instructions,
    model="gpt-4o-mini")

recipe_details_tool = recipe_details_agent.as_tool(
    tool_name="recipe_details_agent",
    tool_description="Recipe details agent tool")

In [49]:

subject_instructions = "You are a subject line writer for a recipe email. \
You write a subject line for a recipe email that is engaging and relevant to the recipe."

subject_agent = Agent(
    name="Subject Line Writer",
    instructions=subject_instructions,
    model=gemini_model)

subject_tool = subject_agent.as_tool("subject_agent", "Write a subject line for a recipe email")

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_agent = Agent(
    name="HTML Converter",
    instructions=html_instructions,
    model=gemini_model)

html_tool = html_agent.as_tool("html_agent", "Convert a text email body to an HTML email body")

In [ ]:
sender_email_id=os.getenv('SENDER_EMAIL_ID')
receiver_email_id=os.getenv('RECEIVER_EMAIL_ID')

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email(sender_email_id)  # Change to your verified sender
    to_email = To(receiver_email_id)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [66]:
tools = [subject_tool, html_tool, send_html_email]

emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    model=gemini_model,
    instructions = emailer_instructions,
    tools = tools
)

In [ ]:
orchestrator_instructions = """ 
You are the **Recipe of the Day Orchestrator**, a culinary conductor that coordinates several expert cooking agentsand return a single "Recipe of the Day".

### Your mission
Plan and deliver a complete recipe of the day by:
1. Calling the Recipe ideation agent to provide the list of recipes
2. Call the Recipe filter and selector agent passing all the recipe to produce one recipe.
3. Call the Recipe details tool passing single recipe to product recipe details.
4. Handoff for Sending: Pass ONLY the selected recipe to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.

### Rules
- Use sub-agents rather than making up all details yourself.

### Tone
Warm, creative, and professional — like a magazine feature.
"""

recipe_of_a_day = Agent(
    name="Recipe of a day", 
    instructions=orchestrator_instructions,
    tools=[recipe_ideation_tool, recipe_filter_selector_tool, recipe_details_tool],
    handoffs=[emailer_agent],
    model=gemini_model)

message = "Generate recipe of a day"

with trace("Recipe of a day"):
    result = await Runner.run(recipe_of_a_day, message)

print(result.final_output)